In [ ]:
from sample_generator import *
from RobustSTL import *
import numpy as np
import math
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline
from sklearn.preprocessing import MinMaxScaler
from statsmodels.tsa.holtwinters import Holt
import statsmodels.formula.api as smf
import json


In [ ]:
df = pd.read_csv("../data/demo_data.csv")
df

In [ ]:
data = df.iloc[:, 1:].values
print(data.shape)

In [ ]:
select_idx = 0

In [ ]:
select_data = data[-15*288:, select_idx]

In [ ]:
# 归一化
scaler = MinMaxScaler()
normalized_data = scaler.fit_transform(select_data.reshape(-1, 1))

# # 反归一化
# recovered_data = scaler.inverse_transform(normalized_data)

In [ ]:
print(normalized_data.shape)

In [ ]:
train_data = normalized_data[:14*288, :]
test_data = normalized_data[-288:, :]
print(train_data.shape, test_data.shape)

In [ ]:
result = RobustSTL(train_data, 288, reg1=10.0, reg2= 0.5, K=2, H=5, ds1=10)

In [ ]:
fig = plt.figure(figsize=(30,25))
import matplotlib
matplotlib.rcParams.update({'font.size': 22})

samples = zip(result, ['sample', 'trend', 'seasonality', 'remainder'])

for i, item in enumerate(samples):
    plt.subplot(4,1,(i+1))
    if i==0:
        plt.plot(item[0], color='blue')
        plt.title(item[1])
        plt.subplot(4,1,i+2)
        plt.plot(item[0], color='blue')
    else:
        plt.plot(item[0], color='red')
        plt.title(item[1])
    
plt.show()

In [ ]:
# 趋势项预测
trend_data = result[1]
hs_1 = Holt(trend_data).fit()
# 残差项预测
remainder_data = pd.DataFrame({'ts':range(len(result[3])),'qps':result[3]})
mod = smf.quantreg('qps ~ ts', remainder_data)
res = mod.fit(q=.99)
print(res.summary())
# 预测
trend_forecast = hs_1.forecast(288)
period_forecast = result[2][-288:]
remainder_forecast = [i*res.params['ts']+res.params['Intercept'] for i in range(4321,4321+288)]
# data_forecast = trend_forecast+period_forecast+remainder_forecast
data_forecast = trend_forecast+period_forecast

In [ ]:
fig = plt.figure(figsize=(15,6))
plt.plot(data_forecast, label = 'pred')
plt.plot(test_data, label = 'truth')
plt.legend()
plt.show()

In [ ]:
recovered_forecast_data = scaler.inverse_transform(data_forecast.reshape(-1, 1))
recovered_test_data = scaler.inverse_transform(test_data.reshape(-1, 1))
fig = plt.figure(figsize=(15,6))
plt.plot(recovered_forecast_data, label = 'pred')
plt.plot(recovered_test_data, label = 'truth')
plt.legend()
plt.show()

In [ ]:
pred_list = recovered_forecast_data.tolist()
with open('performance_table_QT.json', 'w', encoding='utf-8') as f:
    # 将字典保存为JSON
    json.dump(table, f, ensure_ascii=False, indent=4)